# calculating MPC(Mean Phase Coherence) as a biomarker for DMN

In [1]:
#########phase coherence in a frequecy band ,f e [4-8]hz#########

import numpy as np

def PhaseCoherence(freq1,freq2, timeSeries, FS):

    # Get parameters of input data
    nMeasures = np.shape(timeSeries)[0]
    nSamples = np.shape(timeSeries)[1]
    nSecs = nSamples / FS
    
    # Calculate FFT for each measurement (spect is freq x measurements)
    spect = np.fft.fft(timeSeries, axis=1)
    
    # Normalise by amplitude
    spect = spect / abs(spect)
    
    # Find spectrum values for frequency bin of interest
    freqRes = 1 / nSecs;
    foibin1 = round(freq1 / freqRes + 1) - 1
    foibin2 = round(freq2 / freqRes + 1) - 1
    spectFoi = spect[:,foibin1:(foibin2+1)]
    
    # Find individual phase angles per measurement at frequency of interest
    anglesFoi = np.arctan2(spectFoi.imag, spectFoi.real)
    
    # PC is root mean square of the sums of the cosines and sines of the angles
    PC = np.sqrt((np.sum(np.cos(anglesFoi)))**2 + (np.sum(np.sin(anglesFoi)))**2) / np.shape(anglesFoi)[0]
    
    # Print the value
    print('----------------------------------');
    print('Phase coherence value = ' + str("{0:.3f}".format(PC)));
        
    return PC

#DIVIDING ELECTRODES TO GROUPS
# left frontal (Fl)=fp1,f3,f7=0,2,10
# right frontal (Fr)=fp2,f4,f8=1,3,11
# left central (C)=t3, c3=12,4
# right central (C)=t4, c4=13,5
# left parietal-occipital (lPO)=t5,p3,o1=14,6,8
# right parietal-occipital (rPO)=t6,p4,o2=15,7,9
# midline (M)=fz,cz,pz=17,16,18
#alpha=[8 13]
#gamma=[25 45]
#theta=[4 8]
#right theta
# right frontal (Fr)=fp2,f4,f8=1,3,11===/0 1 2
# right central (C)=t4, c4=13,5==/3 4
# right parietal-occipital (rPO)=t6,p4,o2=15,7,9==/5 6 7
# midline (M)=fz,cz,pz=17,16,18==/8 9 10
#Fr to C=(0,3)(0,4)(1,3)(1,4)(2,3)(2,4)
#Fr to rPO=(0,5)(0,6)(0,7)(1,5)(1,6)(1,7)(2,5)(2,6)(2,7)
#Fr to M=(0,8)(0,9)(0,10)(1,8)(1,9)(1,10)(2,8)(2,9)(2,10)
#rPO to C=(5,3)(5,4)(6,3)(6,4)(7,3)(7,4)
#M to C=(8,3)(8,4)(9,3)(9,4)(10,3)(10,4)
#M to rPO=(8,5)(8,6),(8,7),(9,5)(9,6),(9,7)(10,5)(10,6)(10,7)

In [2]:
import numpy as np
freq1=4
freq2=8
FS=250
score=np.empty([1,24])
subject=['a','b','c','d','e','f','g','h','i','j','k','m','n','o','p','q','r','s','t','u','w','x','y','z']
for ii in [0,23]:
    data_raw = np.load('D:\\jupyter_folder\\nurun_python_codes\\pre_S'+f'{subject[ii]}''.npy') 
    eeg=data_raw[[1,3,11,13,5,15,7,9,17,16,18],:]
    #data_raw = scipy.io.loadmat('E:\software\EEG+Neuroguide report\EEG_Mat\S'+f'{subject[ii]}''.mat')
    #f=data_raw['EEG_Data']
    #eeg=np.array((f))
    del data_raw
    PC=np.empty([11,11])
    for i1 in range(0,10):
        data=np.empty
        eeg1=eeg[i1,:]
        for i2 in range(0,10):
            eeg2=eeg[i2,:]
            data=[eeg1,eeg2]
            del eeg2
            PC[i1,i2] = PhaseCoherence(freq1,freq2, data, FS)
            del data
    del eeg1  
    del eeg
    Fr_C=(PC[0,3]+PC[0,4]+PC[1,3]+PC[1,4]+PC[2,3]+PC[2,4])/6
    Fr_rPO=(PC[0,5]+PC[0,6]+PC[0,7]+PC[1,5]+PC[1,6]+PC[1,7]+PC[2,5]+PC[2,6]+PC[2,7])/9
    Fr_M=(PC[0,8]+PC[0,9]+PC[0,10]+PC[1,8]+PC[1,9]+PC[1,10]+PC[2,8]+PC[2,9]+PC[2,10])/9
    rPO_C=(PC[5,3]+PC[5,4]+PC[6,3]+PC[6,4]+PC[7,3]+PC[7,4])/6
    M_C=(PC[8,3]+PC[8,4]+PC[9,3]+PC[9,4]+PC[10,3]+PC[10,4])/6
    M_rPO=(PC[8,5]+PC[8,6]+PC[8,7]+PC[9,5]+PC[9,6]+PC[9,7]+PC[10,5]+PC[10,6]+PC[10,7])/6
    sco=(Fr_C + Fr_rPO + Fr_M + rPO_C + M_C + M_rPO)/6
    del Fr_C
    del Fr_rPO
    del Fr_M
    del rPO_C
    del M_C
    del M_rPO
    score[0][ii]=sco
    del sco
    z_score=np.arctanh(score)

----------------------------------
Phase coherence value = 6.995
----------------------------------
Phase coherence value = 13.118
----------------------------------
Phase coherence value = 14.748
----------------------------------
Phase coherence value = 13.667
----------------------------------
Phase coherence value = 13.854
----------------------------------
Phase coherence value = 14.772
----------------------------------
Phase coherence value = 17.484
----------------------------------
Phase coherence value = 12.571
----------------------------------
Phase coherence value = 14.319
----------------------------------
Phase coherence value = 12.762
----------------------------------
Phase coherence value = 13.118
----------------------------------
Phase coherence value = 19.247
----------------------------------
Phase coherence value = 20.880
----------------------------------
Phase coherence value = 18.756
----------------------------------
Phase coherence value = 19.973
-----------

<ipython-input-2-4ce21eb5b093>:41: RuntimeWarning: invalid value encountered in arctanh
  z_score=np.arctanh(score)


----------------------------------
Phase coherence value = 15.583
----------------------------------
Phase coherence value = 17.330
----------------------------------
Phase coherence value = 16.306
----------------------------------
Phase coherence value = 15.433
----------------------------------
Phase coherence value = 17.588
----------------------------------
Phase coherence value = 11.733
----------------------------------
Phase coherence value = 7.997
----------------------------------
Phase coherence value = 9.083
----------------------------------
Phase coherence value = 12.316
----------------------------------
Phase coherence value = 16.016
----------------------------------
Phase coherence value = 17.330
----------------------------------
Phase coherence value = 19.347
----------------------------------
Phase coherence value = 17.930
----------------------------------
Phase coherence value = 16.969
----------------------------------
Phase coherence value = 15.901
------------

In [3]:
print(score)
print(z_score)

[[1.74715758e+001 3.56043053e-307 1.60219306e-306 7.56571288e-307
  1.89146896e-307 1.37961302e-306 1.05699242e-307 8.01097889e-307
  1.78020169e-306 7.56601165e-307 1.02359984e-306 1.33510679e-306
  2.22522597e-306 1.33511018e-306 6.23057689e-307 1.33511290e-306
  1.78019082e-306 8.45559303e-307 8.06613040e-308 6.89810244e-307
  1.22387550e-307 2.22522596e-306 8.34423917e-308 1.37261740e+256]]
[[            nan 3.56043053e-307 1.60219306e-306 7.56571288e-307
  1.89146896e-307 1.37961302e-306 1.05699242e-307 8.01097889e-307
  1.78020169e-306 7.56601165e-307 1.02359984e-306 1.33510679e-306
  2.22522597e-306 1.33511018e-306 6.23057689e-307 1.33511290e-306
  1.78019082e-306 8.45559303e-307 8.06613040e-308 6.89810244e-307
  1.22387550e-307 2.22522596e-306 8.34423917e-308             nan]]


In [26]:
#########phase coherence in an absoulute frequecy,f e 3hz#########
import numpy as np
def PhaseCoherence(freq, timeSeries, FS):

    # Get parameters of input data
    nMeasures	 = np.shape(timeSeries)[0]
    nSamples 	= np.shape(timeSeries)[1]
    nSecs = nSamples / FS
    print('Number of measurements =', nMeasures)
    print('Number of time samples =', nSamples, '=', nSecs, 'seconds')
    
    # Calculate FFT for each measurement (spect is freq x measurements)
    spect = np.fft.fft(timeSeries, axis=1)
    
    # Normalise by amplitude
    spect = spect / abs(spect)
    
    # Find spectrum values for frequency bin of interest
    freqRes = 1 / nSecs;
    foibin = round(freq / freqRes + 1) - 1
    spectFoi = spect[:,foibin]
    
    # Find individual phase angles per measurement at frequency of interest
    anglesFoi = np.arctan2(spectFoi.imag, spectFoi.real)
    
    # PC is root mean square of the sums of the cosines and sines of the angles
    PC = np.sqrt((np.sum(np.cos(anglesFoi)))**2 + (np.sum(np.sin(anglesFoi)))**2) / np.shape(anglesFoi)[0]
    
    # Print the value
    print('----------------------------------');
    print('Phase coherence value = ' + str("{0:.3f}".format(PC)));
        
    return PC

In [ ]:
1.38266715120789	1.35828411306608	1.39731271180647	1.45631365718719	1.51796450093155	1.72427597845290	1.49743417767631	1.47733112684247	1.45242131667413	1.58319059752991	1.54406814741389	1.38858216570681	1.50860900509054	1.35124089123607	1.37019223815778	1.45719450413031	1.45160956942187	1.44954731652005	1.45348550105287	1.47200126276418	1.40402647251869	1.43085544399284	1.35280673489043	1.41638651095603